## Métrica desconocimiento

In [1]:
from dbconn import querydb, querydbtopandas 
import pandas as pd 

In [2]:
#preguntar a que se refiere con id_tipo en encuestas
#preguntar xq tenemos prestador, efector, demandante 

In [3]:
mayor = "2019-12"
menor = "2019-10"

In [4]:
df = querydbtopandas(f"""select    a11.id_encuesta  id_encuesta,
                
                a13.id_pre_tipo,

                a11.id_afiliado  id_afi_afiliado,

                a11.id_prestador  id_pre_prestador,
                
                a16.id_pre_prestador  id_prestador_efector,

                max(a13.desc_pre_nombre)  desc_pre_nombre,

                max(a13.vigente)  vigente,

                a11.id_transac_tipo  id_transac_tipo,

                a11.id_prest_efector  id_pre_presador_efector,

                max(a16.desc_pre_nombre)  desc_pre_nombre_efector,

                a11.id_prest_prestacion  id_pres_prestacion,

                max(a18.desc_pres_prestacion)  desc_pres_prestacion,

                max(a18.prestac_pres_prestacion)  prestac_pres_prestacion,

                sum(distinct a11.i_CNT_desc)  QDESCONOCIDAS

from      DBA.ft_prest_encuestas              a11

                join        DBA.d_pre_prestador   a13

                  on        (a11.id_prestador = a13.id_pre_prestador)

                join        DBA.d_pre_prestador_efector  a16

                  on        (a11.id_prest_efector = a16.id_pre_prestador)

                join        DBA.d_pres_prestacion a18

                  on        (a11.id_prest_prestacion = a18.id_pres_prestacion)


where (cast(a11.fec_encuesta as date) >= date("{menor}") and cast(a11.fec_encuesta as date) < date("{mayor}")

and a11.id_moti_desconoc <> 1)

group by             a11.id_encuesta,

                    a13.id_pre_tipo,

                a11.id_afiliado,

                a11.id_prestador,
                
                 a16.id_pre_prestador, 

                a13.id_pre_especialidad,

                a11.id_transac_tipo,

                a11.id_prest_efector,

                a11.id_prest_prestacion,

                DATE(a11.fec_consumo),

                a11.id_tie_mes_c""")

In [5]:
df.head()

,id_encuesta,id_pre_tipo,id_afi_afiliado,id_pre_prestador,id_prestador_efector,desc_pre_nombre,vigente,id_transac_tipo,id_pre_presador_efector,desc_pre_nombre_efector,id_pres_prestacion,desc_pres_prestacion,prestac_pres_prestacion,QDESCONOCIDAS
0,545345,P,3552252,20568,0,"Aman,Fermín E.",A,0,0,Sin Asignar,2061,Consulta medica en consultorio,42010100,1.0
1,560164,0,0,0,0,Sin Asignar,B,0,0,Sin Asignar,0,No Asignado,0,1.0
2,566281,0,0,0,0,Sin Asignar,B,0,0,Sin Asignar,0,No Asignado,0,1.0
3,566573,P,3143003,73533,0,"Ostrovsky,Mariano E.",A,1,0,Sin Asignar,2061,Consulta medica en consultorio,42010100,1.0
4,554013,I,3296166,31868,0,Clínica Privada Modelo de Río Tercero,A,0,0,Sin Asignar,2061,Consulta medica en consultorio,42010100,1.0


In [6]:
clinicas_propias_id = [1632,1763,2086,26564,55060 ,64677,75470,78363 ,85316,85401 ,99785 ,118209 ,122803,135464 ,182730,187243 ,196559 ,196561 ,206939 ,210252 ,211421, 213729 ,217747 ,226756,235779,240895] 
cma = df[df["id_pre_prestador"].isin(clinicas_propias_id)][["id_pre_prestador", "desc_pre_nombre"]].rename(columns={"desc_pre_nombre": "cma"})
df = pd.merge(df, cma, how="outer").fillna("No aplica")

In [7]:
propios = df.cma.value_counts().index[1:]

In [8]:
#df_part_equipos = df[df["id_pre_tipo"].isin(["P","E"])][["id_encuesta", "id_afi_afiliado", "id_pre_prestador", "desc_pre_nombre", "cma" ]]
#df_inst_circ = df[df["id_pre_tipo"].isin(["I","C"])][["id_encuesta","id_afi_afiliado", "id_prestador_efector", 'desc_pre_nombre_efector', "cma"]]
#df_inst_circ.rename(columns={"id_prestador_efector": "id_pre_prestador", "desc_pre_nombre_efector":"desc_pre_nombre"},inplace=True)

In [9]:
#df = pd.concat([df_part_equipos, df_inst_circ])

In [10]:
#df[df.cma.isin(propios)]

In [11]:
df = df[df["id_pre_prestador"] != 0]

In [12]:
df.cma.value_counts()

No aplica                                           1885
SWISS MEDICAL CENTER BARRIO NORTE                    900
CLINICA ZABALA                                       484
SWISS MEDICAL CENTER MICROCENTRO SEDE 25 DE MAYO     400
SWISS MEDICAL CENTER LOMAS                           256
SWISS MEDICAL CENTER DE LOS ARCOS                    196
SANATORIO DE LOS ARCOS                               196
CLINICA OLIVOS                                        49
CLINICA Y MATERNIDAD SUIZO ARGENTINA                  49
SWISS MEDICAL CENTER MICROCENTRO SEDE SARMIENTO       25
SWISS MEDICAL CENTER BERAZATEGUI                       9
SWISS MEDICAL CENTER BARRIO PARQUE                     4
SWISS MEDICAL CENTER OLIVOS                            4
SWISS MEDICAL CENTER ALMAGRO                           1
Name: cma, dtype: int64

In [13]:
df.cma.unique()

array(['No aplica', 'SWISS MEDICAL CENTER DE LOS ARCOS', 'CLINICA OLIVOS',
       'SWISS MEDICAL CENTER LOMAS', 'SWISS MEDICAL CENTER OLIVOS',
       'SWISS MEDICAL CENTER MICROCENTRO SEDE 25 DE MAYO',
       'CLINICA ZABALA', 'SWISS MEDICAL CENTER BARRIO NORTE',
       'SANATORIO DE LOS ARCOS',
       'SWISS MEDICAL CENTER MICROCENTRO SEDE SARMIENTO',
       'SWISS MEDICAL CENTER BERAZATEGUI',
       'CLINICA Y MATERNIDAD SUIZO ARGENTINA',
       'SWISS MEDICAL CENTER ALMAGRO',
       'SWISS MEDICAL CENTER BARRIO PARQUE'], dtype=object)

In [14]:
#count de encuestas
encuestas_desconocidas = df.groupby(["id_pre_prestador","desc_pre_nombre","cma"])["id_encuesta"].nunique()

In [15]:
#count de afiliados
afiliados_desconocidos = df.groupby(["id_pre_prestador","desc_pre_nombre", "cma"])["id_afi_afiliado"].nunique()

In [16]:
df = pd.concat([encuestas_desconocidas, afiliados_desconocidos],axis=1).reset_index()

In [17]:
df

,id_pre_prestador,desc_pre_nombre,cma,id_encuesta,id_afi_afiliado
0,105,"Cohen,Diego E.",No aplica,10,3
1,172,"Pasqualini,Rodolfo S.",No aplica,2,2
2,232,"Fioroni,Andrea P.",No aplica,1,1
3,346,"Silberman,Andrés A.",No aplica,2,1
4,399,"De Elizalde,Santiago",No aplica,1,1
...,...,...,...,...,...
610,234977,"Lacey,Paola M.",No aplica,1,1
611,235798,Centro Médico Consejo Salud,No aplica,2,2
612,238913,"Rueda,Adriana S.",No aplica,2,1
613,239491,"Zapata,María Angélica",No aplica,1,1


In [18]:
df.rename(columns = {"id_pre_prestador":"Prestador",
                     "desc_pre_nombre":"Razón Social",
                     "id_encuesta":"Q desconocidas",
                     "id_afi_afiliado":"Q afiliados"},inplace=True)

In [19]:
df.columns

Index(['Prestador', 'Razón Social', 'cma', 'Q desconocidas', 'Q afiliados'], dtype='object')

In [20]:
df = df[['Prestador', 'Razón Social', 'cma', 'Q afiliados', 'Q desconocidas']]

In [22]:
oct_nov_desc = df.sort_values(by= "Q afiliados", ascending = False).head(100)

In [25]:
oct_nov_desc.to_excel("desc_oct_nov.xlsx")

In [216]:
final_1

,Prestador,Razón Social,cma,Q desconocidas,Q afiliados
0,105,"Cohen,Diego E.",No aplica,9,2
1,172,"Pasqualini,Rodolfo S.",No aplica,1,1
2,432,"Juarez Goñi,Francisco",No aplica,1,1
3,529,"Schvartzman,Javier A.",No aplica,1,1
4,636,"Cahué,Jorge",No aplica,2,1
...,...,...,...,...,...
211,215973,"Davin,Florencia",No aplica,1,1
212,219030,"Tagliabue,Marcela",No aplica,1,1
213,219373,"Striebeck,Miranda E.",No aplica,1,1
214,238913,"Rueda,Adriana S.",No aplica,2,1
